# Acquire

In [1]:
import requests
import numpy as np
import pandas as pd


from bs4 import BeautifulSoup
from requests import get

## LinkedIn Job Postings

In [2]:
# Enter user-agent credentials and url into reponse object.
params = {'User-Agent': 'Blue-Owl-Data'}
url = 'https://www.linkedin.com/jobs/search/?keywords=data%20science'
response = get(url, params)

# Create a beautiful soup object of job postings in html format.
soup = BeautifulSoup(response.content, 'html.parser')

# Display the number of job postings collected.
len(soup.find_all('h3', class_="result-card__title job-result-card__title"))

24

## Job Title

In [3]:
for job in soup.find_all('h3', class_="result-card__title job-result-card__title"):
    print(job.text)

Applied Research Data Science Intern
Data science intern
Junior Data Scientist
Data Science Intern
Data Science Intern
Data Scientist (ML), Internship
Data Science Intern
Junior Data Scientist
Data Scientist
Data Scientist Intern
Data & Applied Scientist
Product Data Scientist - Ads Data Science
Data Science Intern
Data Scientist, Population Health (Boston, MA)
Intern- Data Science
Data Scientist
Intern - Data Analytics
Intern: Biostatistics, Analytics and Data Sciences I
Data Science Co-op
Product Analyst, Data Science
2021 Intern, Data Science
Junior Data Scientist (Portland or NY based)
Intern - Data Scientist
Data Science/Analytics Paid Internship


## Location

In [4]:
for location in soup.find_all('span', class_="job-result-card__location"):
    print(location.text)

Sunnyvale, CA
San Jose, CA
Austin, TX
San Diego, CA
San Jose, CA
San Francisco, CA
Seattle, WA
Livonia, MI
Cupertino, CA
San Diego, CA
Redmond, WA
New York, NY
San Jose, CA
Boston, MA
Foster City, CA
San Francisco, CA
Palo Alto, CA
Cambridge, MA
Indianapolis, IN
New York, NY
Remote, OR
Portland, OR
Santa Clara, CA
Chicago, IL


## Company Name

In [5]:
for i in soup.find_all('h4', class_="result-card__subtitle job-result-card__subtitle"):
    print(i.text)

LinkedIn
PayPal
Analytos
Intel Corporation
Adobe
DoorDash
Amazon
Soothsayer Analytics
Apple
PlayStation
Microsoft
Twitter
Roku Inc.
Verily Life Sciences
Gilead Sciences
Notion
Stockpile
Biogen
Delta Faucet Company
Google
Waymo
Mars
Palo Alto Networks
Chubb


## Original Job Posting Date

In [26]:
for i in soup.find_all('time', class_="job-result-card__listdate"):
    print(i['datetime'])

2021-01-16
2021-01-15
2021-01-16
2020-12-25
2021-01-15
2020-12-24
2021-01-17
2021-01-16
2021-01-16
2021-01-17
2021-01-17
2021-01-16
2021-01-12
2021-01-14
2020-11-16
2021-01-12
2021-01-17
2021-01-06


## Job Post Age

In [6]:
for age in soup.find_all('time', class_="job-result-card__listdate"):
    print(age.text)

2 days ago
4 days ago
3 days ago
3 weeks ago
3 days ago
4 weeks ago
2 days ago
2 days ago
2 days ago
1 day ago
1 day ago
2 days ago
6 days ago
5 days ago
2 months ago
7 days ago
2 days ago
2 weeks ago


## Link

In [10]:
counter = 0

for link in soup.find_all('a', class_="result-card__subtitle-link job-result-card__subtitle-link"):
    print(link['href'])

https://www.linkedin.com/company/linkedin?trk=public_jobs_job-result-card_result-card_subtitle-click
https://www.linkedin.com/company/paypal?trk=public_jobs_job-result-card_result-card_subtitle-click
https://www.linkedin.com/company/analytos?trk=public_jobs_job-result-card_result-card_subtitle-click
https://www.linkedin.com/company/intel-corporation?trk=public_jobs_job-result-card_result-card_subtitle-click
https://www.linkedin.com/company/adobe?trk=public_jobs_job-result-card_result-card_subtitle-click
https://www.linkedin.com/company/doordash?trk=public_jobs_job-result-card_result-card_subtitle-click
https://www.linkedin.com/company/amazon?trk=public_jobs_job-result-card_result-card_subtitle-click
https://www.linkedin.com/company/soothsayer-analytics?trk=public_jobs_job-result-card_result-card_subtitle-click
https://www.linkedin.com/company/apple?trk=public_jobs_job-result-card_result-card_subtitle-click
https://www.linkedin.com/company/playstation-sony?trk=public_jobs_job-result-car

## Description

In [39]:
for i in soup.find_all('p', class_="job-result-card__snippet"):
    print(i.text)

Ideally this individual will understand the fundamentals of experimentation, analytic thinking and have experience with modeling data in ...
The AI & Machine Learning R&D Team in PayPal is looking for talented applied research intern to work on machine learning research for ...
If you want to make meaningful contributions and reinvent the retail space that will impact millions of guests, Target is for you.About ...
1+ year experience in algorithmic implementation using a high-level, modern programming language such as C++ or Python. U.S. experience ...
What you will do. Discover what our employees are saying about their career experiences on the Adobe Life blog and explore the meaningful...
High-energy and confident — you keep the mission in mind, take ideas and help them grow using data and rigorous testing, show evidence ...
We are looking for motivated data scientists with excellent leadership skills, and the ability to develop, automate, and run analytical ...
Our major clients inc

In [16]:
print(soup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta content="d_jobs_guest_search" name="pageKey"/>
  <meta content="urlType=jserp_custom;emptyResult=false" name="linkedin:pageTag"/>
  <meta content="en_US" name="locale"/>
  <meta data-app-id="com.linkedin.jobs-guest-frontend.d_web" data-custom-tracking-code="" data-jet-tags="" data-service-name="jobs-guest-frontend" data-tracking-page-type="" id="config">
   <meta data-use-location-for-dp="" data-use-page-title="" id="google-analytics-config">
    <link href="https://www.linkedin.com/jobs/data-science-jobs" rel="canonical"/>
    <link crossorigin="use-credentials" href="/homepage-guest/manifest.json" rel="manifest">
     <link href="https://static-exp1.licdn.com/sc/h/al2o9zrvru7aqj8e1x2rzsrca" rel="icon"/>
     <script>
      function getDfd() {let yFn,nFn;const p=new Promise((y, n)=>{yFn=y;nFn=n;});p.resolve=yFn;p.reject=nFn;return p;}window.lazyloader = getDfd();window.tracking = getDfd();window.impressionTracking = getDfd();window.pemT

# LinkedIn Job Posting Dataset